In [3]:
"""
Unified IDP Analysis: Nardini+ features, CIDER parameters, and SPARROW parameters
All in one environment, all in one output file
Uses the proven Nardini calculation method with added CIDER and SPARROW features

FEATURES INCLUDED:
- Nardini: 54 features (raw + z-score) - compositional, physicochemical, patches
- CIDER: 6 features - kappa, omega, delta, mean_net_charge, uversky_hydropathy, fraction_neutral
- SPARROW: 7 features - SCD, SHD, complexity, fraction_proline,
           scaled_rg, scaled_re, prefactor, scaling_exponent, asphericity

Z-SCORE HANDLING:
- Z-scores set to NaN when raw value is 0 (division by zero protection)
- Z-scores set to NaN when reference std dev is near-zero

REDUNDANCY REMOVED:
- Eliminated duplicate features between Nardini, CIDER, and SPARROW
- Use Nardini values for: FCR, NCPR, length, fraction_positive, fraction_negative, fraction_expanding,
  fraction_polar, fraction_aliphatic, fraction_aromatic, ppii_propensity
"""

import numpy as np
import pandas as pd
import re
from localcider.sequenceParameters import SequenceParameters as CIDER_SP
from sparrow import Protein


In [5]:

# ============================================================================
# USER INPUTS
# ============================================================================

input_fasta_file = 'test_seqs.fasta'
output_filename = 'test_idp_features'
filetype = "csv"  # "csv" or "excel"

# Z-score calculation (only for single-species analysis)
calculate_zscores = True
reference_species = "Saccharomyces cerevisiae"
# Options: "Homo sapiens", "Mus musculus", "Rattus norvegicus", "Xenopus tropicalis",
# "Drosophila melanogaster", "Danio rerio", "Saccharomyces cerevisiae",
# "Caenorhabditis elegans", "Arabidopsis thaliana"

# ============================================================================
# LOAD REFERENCE DATA FOR Z-SCORES
# ============================================================================

if calculate_zscores:
    print("="*70)
    print("Z-score calculation enabled.")
    print(f"Loading reference data for: {reference_species}")
    print("="*70)
    
    sheetID = '1yxt0R1G0gdI2bGpjYXgk_h7-1qA6EY6J'
    worksheetName = reference_species.split(" ")[1]
    currurl = f'https://docs.google.com/spreadsheets/d/{sheetID}/gviz/tq?tqx=out:csv&sheet={worksheetName}'
    
    try:
        speciesdf = pd.read_csv(currurl)
        meanvals_species = speciesdf['Mean'].values
        stdvals_species = speciesdf['Std'].values
        mycompfeats_all = speciesdf['Feature'].tolist()
        print(f"Successfully loaded {len(mycompfeats_all)} Nardini features.\n")
    except Exception as e:
        raise ValueError(f"Failed to load reference data: {e}")
else:
    print("="*70)
    print("Z-score calculation disabled.")
    print("="*70 + "\n")
    
    aas = 'ACDEFGHIKLMNPQRSTVY'
    mycompfeats_all = ['fracA', 'fracC', 'fracD', 'fracE', 'fracF', 'fracG', 'fracH', 'fracI',
                        'fracK', 'fracL', 'fracM', 'fracN', 'fracP', 'fracQ', 'fracR', 'fracS',
                        'fracT', 'fracV', 'fracW', 'fracY', 'fracpos', 'fracneg', 'fracpol',
                        'fracali', 'fracaro', 'fracRtoK', 'fracEtoD', 'fracexp', 'fcr', 'ncpr',
                        'mhydro', 'dispro', 'isopoi', 'ppii']
    for aa in aas:
        mycompfeats_all.append(f'patch{aa}')
    mycompfeats_all.append('patchRG')

# ============================================================================
# LOAD FASTA FILE
# ============================================================================

print(f"Loading FASTA file: {input_fasta_file}")
with open(input_fasta_file, 'r') as myfile:
    Lines = myfile.readlines()

subseqs = []
subnames = []
thisseq = ''

for line in Lines:
    cleanline = line.strip()
    if cleanline.startswith('>'):
        subnames.append(cleanline[1:])
        if thisseq != '':
            subseqs.append(thisseq.upper())
            thisseq = ''
    else:
        thisseq += cleanline

subseqs.append(thisseq.upper())
print(f"Loaded {len(subseqs)} sequences\n")

# ============================================================================
# CALCULATE NARDINI+ COMPOSITIONAL FEATURES
# ============================================================================

numInt = 2
minBlockLen = 4
aas = 'ACDEFGHIKLMNPQRSTVY'

# Initialize feature lists (Nardini method)
fracA, fracC, fracD, fracE, fracF, fracG, fracH, fracI, fracK, fracL = [], [], [], [], [], [], [], [], [], []
fracM, fracN, fracP, fracQ, fracR, fracS, fracT, fracV, fracW, fracY = [], [], [], [], [], [], [], [], [], []
fracpos, fracneg, fracpol, fracali, fracaro = [], [], [], [], []
fracRtoK, fracEtoD, fracexp, fcr, ncpr = [], [], [], [], []
mhydro, dispro, isopoi, ppii = [], [], [], []
fracpatch = [[] for _ in range(len(aas))]
rgpatch = []

print("Calculating Nardini+ compositional features...")

for currseq in subseqs:
    if len(currseq) >= 1 and not any(x in currseq for x in "XUZJBO"):
        SeqOb = CIDER_SP(currseq)
        slen = SeqOb.get_length()
        aafrac = SeqOb.get_amino_acid_fractions()

        # Basic sequence properties
        fracexp.append(SeqOb.get_fraction_expanding())
        fcr.append(SeqOb.get_FCR())
        ncpr.append(SeqOb.get_NCPR())
        mhydro.append(SeqOb.get_mean_hydropathy())
        dispro.append(SeqOb.get_fraction_disorder_promoting())
        isopoi.append(SeqOb.get_isoelectric_point())
        ppii.append(SeqOb.get_PPII_propensity(mode='hilser'))

        # Individual AA fractions
        fracA.append(aafrac['A'])
        fracC.append(aafrac['C'])
        fracD.append(aafrac['D'])
        fracE.append(aafrac['E'])
        fracF.append(aafrac['F'])
        fracG.append(aafrac['G'])
        fracH.append(aafrac['H'])
        fracI.append(aafrac['I'])
        fracK.append(aafrac['K'])
        fracL.append(aafrac['L'])
        fracM.append(aafrac['M'])
        fracN.append(aafrac['N'])
        fracP.append(aafrac['P'])
        fracQ.append(aafrac['Q'])
        fracR.append(aafrac['R'])
        fracS.append(aafrac['S'])
        fracT.append(aafrac['T'])
        fracV.append(aafrac['V'])
        fracW.append(aafrac['W'])
        fracY.append(aafrac['Y'])

        # Physicochemical properties
        fracpos.append(aafrac['K'] + aafrac['R'])
        fracneg.append(aafrac['D'] + aafrac['E'])
        fracpol.append(aafrac['Q'] + aafrac['N'] + aafrac['S'] + aafrac['T'] + aafrac['G'] + aafrac['C'] + aafrac['H'])
        fracali.append(aafrac['A'] + aafrac['L'] + aafrac['M'] + aafrac['I'] + aafrac['V'])
        fracaro.append(aafrac['F'] + aafrac['W'] + aafrac['Y'])

        # AA ratios
        fracRtoK.append(np.log10(((slen * aafrac['R']) + 1) / ((slen * aafrac['K']) + 1)))
        fracEtoD.append(np.log10(((slen * aafrac['E']) + 1) / ((slen * aafrac['D']) + 1)))

        # Patches for each amino acid
        for idx, aa in enumerate(aas):
            justKs = '0' * len(currseq)
            pos = [i for i, ltr in enumerate(currseq) if ltr == aa]
            
            pos2 = pos.copy()
            for p in range(len(pos) - 1):
                tdi = pos[p + 1] - pos[p]
                if 1 < tdi <= numInt + 1:
                    pos2.extend(range(pos[p] + 1, pos[p + 1]))
            
            justKs = list(justKs)
            for p in pos2:
                justKs[p] = '1'
            justKs = ''.join(justKs)
            
            the_ones = re.findall(r"1+", justKs)
            idx_ones = [[m.start(0), m.end(0)] for m in re.finditer(r"1+", justKs)]
            
            patchescombined = ''
            for count, o in enumerate(the_ones):
                myrange = idx_ones[count]
                subseq = currseq[myrange[0]:myrange[1]]
                pos3 = [i for i, ltr in enumerate(subseq) if ltr == aa]
                if len(pos3) >= minBlockLen:
                    patchescombined += subseq
            
            fracpatch[idx].append(len(patchescombined) / len(currseq))

        # RG patch calculation
        justKs = '0' * len(currseq)
        pos = [i for i, ltr in enumerate(currseq) if ltr in 'RG']
        
        pos2 = pos.copy()
        for p in range(len(pos) - 1):
            tdi = pos[p + 1] - pos[p]
            if 1 < tdi <= numInt + 1:
                pos2.extend(range(pos[p] + 1, pos[p + 1]))
        
        justKs = list(justKs)
        for p in pos2:
            justKs[p] = '1'
        justKs = ''.join(justKs)
        
        the_ones = re.findall(r"1+", justKs)
        idx_ones = [[m.start(0), m.end(0)] for m in re.finditer(r"1+", justKs)]
        
        patchescombined = ''
        for count, o in enumerate(the_ones):
            myrange = idx_ones[count]
            subseq = currseq[myrange[0]:myrange[1]]
            if subseq.count('RG') >= 2:
                patchescombined += subseq
        
        rgpatch.append(len(patchescombined) / len(currseq))
    
    else:
        # Handle invalid sequences - append NaN to all feature lists
        fracexp.append(np.nan)
        fcr.append(np.nan)
        ncpr.append(np.nan)
        mhydro.append(np.nan)
        dispro.append(np.nan)
        isopoi.append(np.nan)
        ppii.append(np.nan)
        
        # Individual AA fractions
        fracA.append(np.nan)
        fracC.append(np.nan)
        fracD.append(np.nan)
        fracE.append(np.nan)
        fracF.append(np.nan)
        fracG.append(np.nan)
        fracH.append(np.nan)
        fracI.append(np.nan)
        fracK.append(np.nan)
        fracL.append(np.nan)
        fracM.append(np.nan)
        fracN.append(np.nan)
        fracP.append(np.nan)
        fracQ.append(np.nan)
        fracR.append(np.nan)
        fracS.append(np.nan)
        fracT.append(np.nan)
        fracV.append(np.nan)
        fracW.append(np.nan)
        fracY.append(np.nan)
        
        # Physicochemical properties
        fracpos.append(np.nan)
        fracneg.append(np.nan)
        fracpol.append(np.nan)
        fracali.append(np.nan)
        fracaro.append(np.nan)
        
        # AA ratios
        fracRtoK.append(np.nan)
        fracEtoD.append(np.nan)
        
        # Patches
        for idx in range(len(aas)):
            fracpatch[idx].append(np.nan)
        
        rgpatch.append(np.nan)

# Combine all Nardini features
compfeatvals_all = [fracA, fracC, fracD, fracE, fracF, fracG, fracH, fracI, fracK, fracL, 
                    fracM, fracN, fracP, fracQ, fracR, fracS, fracT, fracV, fracW, fracY, 
                    fracpos, fracneg, fracpol, fracali, fracaro, fracRtoK, fracEtoD, 
                    fracexp, fcr, ncpr, mhydro, dispro, isopoi, ppii]

for a in fracpatch:
    compfeatvals_all.append(a)
compfeatvals_all.append(rgpatch)

print("Nardini+ features calculated.\n")

# ============================================================================
# CREATE NARDINI DATAFRAME WITH RAW VALUES AND Z-SCORES
# ============================================================================

num_seqs = len(subseqs)
num_feats = len(mycompfeats_all)
subrawcomp = np.zeros((num_seqs, num_feats))

# Calculate raw values
for s in range(num_seqs):
    for f in range(num_feats):
        subrawcomp[s, f] = compfeatvals_all[f][s]

# Create dataframe with raw values
raw_df = pd.DataFrame(data=subrawcomp, columns=mycompfeats_all)

if calculate_zscores:
    # Calculate z-scores
    print("Calculating z-scores for Nardini features...")
    subzveccomp = np.zeros((num_seqs, num_feats))
    
    min_std_threshold = 1e-10
    zero_std_features = []
    
    for f in range(num_feats):
        if stdvals_species[f] < min_std_threshold:
            zero_std_features.append(mycompfeats_all[f])
    
    if zero_std_features:
        print(f"Warning: {len(zero_std_features)} feature(s) have zero/near-zero std dev")
    
    for s in range(num_seqs):
        for f in range(num_feats):
            raw_val = compfeatvals_all[f][s]
            # Set z-score to NaN if raw value is zero OR if std is too small
            if raw_val == 0 or stdvals_species[f] < min_std_threshold:
                subzveccomp[s, f] = np.nan
            else:
                subzveccomp[s, f] = (raw_val - meanvals_species[f]) / stdvals_species[f]
    
    # Create column names with nardini prefix and suffixes
    raw_cols = [f"nardini_{feat}_raw" for feat in mycompfeats_all]
    zscore_cols = [f"nardini_{feat}_zscore" for feat in mycompfeats_all]
    
    # Rename columns
    raw_df.columns = raw_cols
    
    # Create z-score dataframe
    zscore_df = pd.DataFrame(data=subzveccomp, columns=zscore_cols)
    
    # Merge Nardini features
    nardini_df = pd.concat([raw_df, zscore_df], axis=1)
    print(f"Calculated {num_feats} raw features and {num_feats} z-scores.\n")
    
else:
    # Just use raw values with nardini prefix
    raw_cols = [f"nardini_{feat}_raw" for feat in mycompfeats_all]
    raw_df.columns = raw_cols
    nardini_df = raw_df
    print(f"Calculated {num_feats} raw features (no z-scores).\n")

# ============================================================================
# CALCULATE CIDER AND SPARROW PARAMETERS
# ============================================================================
# REDUNDANT FEATURES REMOVED:
# - CIDER: length, fraction_negative, fraction_positive, fraction_expanding, ppii_propensity
#   (already in Nardini as length, fracneg, fracpos, fracexp, ppii)
# - SPARROW: length, FCR, NCPR, fraction_positive, fraction_negative, fraction_polar,
#   fraction_aliphatic, fraction_aromatic
#   (already in Nardini as length, fcr, ncpr, fracpos, fracneg, fracpol, fracali, fracaro)
# ============================================================================

print("Calculating CIDER and SPARROW parameters...")

cider_data = []
sparrow_data = []

for idx, currseq in enumerate(subseqs):
    if len(currseq) < 1 or any(x in currseq for x in "XUZJBO"):
        # Add NaN rows for invalid sequences
        cider_data.append({
            'cider_kappa': np.nan,
            'cider_omega': np.nan,
            'cider_delta': np.nan,
            'cider_uversky_hydropathy': np.nan,
            'cider_fraction_neutral': np.nan
        })
        sparrow_data.append({
            'sparrow_SCD': np.nan,
            'sparrow_SHD': np.nan,
            'sparrow_complexity': np.nan,
            'sparrow_fraction_proline': np.nan,
            'sparrow_scaled_rg': np.nan,
            'sparrow_scaled_re': np.nan,
            'sparrow_prefactor': np.nan,
            'sparrow_scaling_exponent': np.nan,
            'sparrow_asphericity': np.nan
        })
        continue
    
    # CIDER parameters (redundant features removed)
    try:
        SeqOb = CIDER_SP(currseq)
        cider_data.append({
            'cider_kappa': SeqOb.get_kappa(),
            'cider_omega': SeqOb.get_Omega(),
            'cider_delta': SeqOb.get_delta(),
            'cider_uversky_hydropathy': SeqOb.get_uversky_hydropathy(),
            'cider_fraction_neutral': SeqOb.get_countNeut() / SeqOb.get_length()
            'cider_length': SeqOb.get_length()
        })
    except Exception as e:
        print(f"  Warning: CIDER failed for sequence {idx+1}: {e}")
        cider_data.append({
            'cider_kappa': np.nan,
            'cider_omega': np.nan,
            'cider_delta': np.nan,
            'cider_uversky_hydropathy': np.nan,
            'cider_fraction_neutral': np.nan
            'cider_length': np.nan
        })
    
    # SPARROW parameters (redundant features removed)
    try:
        prot = Protein(currseq)
        
        # Basic SPARROW parameters (only non-redundant ones)
        sparrow_dict = {
            'sparrow_SCD': prot.SCD,
            'sparrow_SHD': prot.SHD,
            'sparrow_complexity': prot.complexity,
            'sparrow_fraction_proline': prot.fraction_proline
        }
        
        # Conformational parameters using prot.predictor methods
        try:
            sparrow_dict['sparrow_scaled_rg'] = prot.predictor.radius_of_gyration(use_scaled=True)
        except Exception as e:
            if idx == 0:  # Only print warning for first sequence
                print(f"  Note: scaled_rg prediction not available: {e}")
            sparrow_dict['sparrow_scaled_rg'] = np.nan
        
        try:
            sparrow_dict['sparrow_scaled_re'] = prot.predictor.end_to_end_distance(use_scaled=True)
        except Exception as e:
            if idx == 0:
                print(f"  Note: scaled_re prediction not available: {e}")
            sparrow_dict['sparrow_scaled_re'] = np.nan
        
        try:
            sparrow_dict['sparrow_prefactor'] = prot.predictor.prefactor()
        except Exception as e:
            if idx == 0:
                print(f"  Note: prefactor prediction not available: {e}")
            sparrow_dict['sparrow_prefactor'] = np.nan
        
        try:
            sparrow_dict['sparrow_scaling_exponent'] = prot.predictor.scaling_exponent()
        except Exception as e:
            if idx == 0:
                print(f"  Note: scaling_exponent prediction not available: {e}")
            sparrow_dict['sparrow_scaling_exponent'] = np.nan
        
        try:
            sparrow_dict['sparrow_asphericity'] = prot.predictor.asphericity()
        except Exception as e:
            if idx == 0:
                print(f"  Note: asphericity prediction not available: {e}")
            sparrow_dict['sparrow_asphericity'] = np.nan
        
        sparrow_data.append(sparrow_dict)
        
    except Exception as e:
        print(f"  Warning: SPARROW failed for sequence {idx+1}: {e}")
        sparrow_data.append({
            'sparrow_SCD': np.nan,
            'sparrow_SHD': np.nan,
            'sparrow_complexity': np.nan,
            'sparrow_fraction_proline': np.nan,
            'sparrow_scaled_rg': np.nan,
            'sparrow_scaled_re': np.nan,
            'sparrow_prefactor': np.nan,
            'sparrow_scaling_exponent': np.nan,
            'sparrow_asphericity': np.nan
        })

cider_df = pd.DataFrame(cider_data)
sparrow_df = pd.DataFrame(sparrow_data)

print("CIDER and SPARROW calculation complete.\n")

# ============================================================================
# COMBINE ALL FEATURES
# ============================================================================

# Create name and sequence dataframes
name_df = pd.DataFrame({'Name': subnames})
seq_df = pd.DataFrame({'Sequence': subseqs})

# Combine everything
final_df = pd.concat([name_df, seq_df, nardini_df, cider_df, sparrow_df], axis=1)

print("="*70)
print("UNIFIED DATAFRAME SUMMARY")
print("="*70)
print(f"Total sequences: {len(final_df)}")
print(f"Total columns: {len(final_df.columns)}")
print(f"  - Nardini raw features: {len([c for c in final_df.columns if 'nardini_' in c and '_raw' in c])}")
if calculate_zscores:
    print(f"  - Nardini z-scores: {len([c for c in final_df.columns if 'nardini_' in c and '_zscore' in c])}")
print(f"  - CIDER parameters: {len([c for c in final_df.columns if c.startswith('cider_')])}")
print(f"  - SPARROW parameters: {len([c for c in final_df.columns if c.startswith('sparrow_')])}")
print("="*70 + "\n")

print("Preview of unified output:")
display(final_df.head())

# ============================================================================
# SAVE OUTPUT FILE
# ============================================================================

if filetype == 'csv':
    output_file = f'{output_filename}.csv'
    final_df.to_csv(output_file, index=False)
elif filetype == 'excel':
    output_file = f'{output_filename}.xlsx'
    final_df.to_excel(output_file, index=False)

print(f'\nSaved unified output to: {output_file}')
print("\n" + "="*70)
print("Analysis complete! All features in one unified dataframe.")
print("="*70)

Z-score calculation enabled.
Loading reference data for: Saccharomyces cerevisiae
Successfully loaded 54 Nardini features.

Loading FASTA file: test_seqs.fasta
Loaded 4 sequences

Calculating Nardini+ compositional features...
Nardini+ features calculated.

Calculating z-scores for Nardini features...
Calculated 54 raw features and 54 z-scores.

Calculating CIDER and SPARROW parameters...
Error importing GPy.
 If trying to run parrot-optimize, make sure to use `pip install idptools-parrot[optimize]`
CIDER and SPARROW calculation complete.

UNIFIED DATAFRAME SUMMARY
Total sequences: 4
Total columns: 124
  - Nardini raw features: 54
  - Nardini z-scores: 54
  - CIDER parameters: 5
  - SPARROW parameters: 9

Preview of unified output:


,Name,Sequence,nardini_Frac A_raw,nardini_Frac C_raw,nardini_Frac D_raw,nardini_Frac E_raw,nardini_Frac F_raw,nardini_Frac G_raw,nardini_Frac H_raw,nardini_Frac I_raw,...,cider_fraction_neutral,sparrow_SCD,sparrow_SHD,sparrow_complexity,sparrow_fraction_proline,sparrow_scaled_rg,sparrow_scaled_re,sparrow_prefactor,sparrow_scaling_exponent,sparrow_asphericity
0,sp|P09442|DH11_GOSHI Late embryogenesis abunda...,MAHFQNQYSAPEVTQTDAYGNPTRRTDEYGNPIPTQETGRGILGIG...,0.041379,0.006897,0.034483,0.075862,0.006897,0.131034,0.075862,0.041379,...,0.786207,-0.583626,5.674386,0.912752,0.068966,34.489186,77.594970,6.640517,0.531007,0.398005
1,tr|Q9FE69|Q9FE69_GLYTO Seed maturation protein...,MQGGKKAGESIKETAANIGASAKAGMEKTKATVQEKAERMTARDPM...,0.141176,0.000000,0.017647,0.058824,0.000000,0.164706,0.058824,0.023529,...,0.817647,0.164884,6.027997,0.865645,0.047059,40.804775,96.208732,6.087533,0.556331,0.420467
2,tr|Q9FNW5|Q9FNW5_GLYTO Seed maturation protein...,MQGGKKAGESIKETAANIGASAKAGMEKTKATVQEKAERMTARDPV...,0.151515,0.000000,0.018182,0.060606,0.006061,0.157576,0.048485,0.018182,...,0.806061,0.452404,5.909211,0.862077,0.036364,39.424046,93.836143,6.117722,0.552114,0.416239
3,tr|Q9FNW6|Q9FNW6_GLYTO Seed maturation protein...,MQGGKKAGESIKETAANIGASAKAGMEKTKATVQEKAERMTARDPM...,0.148810,0.000000,0.017857,0.065476,0.000000,0.160714,0.059524,0.017857,...,0.809524,-0.148899,5.972150,0.856693,0.047619,40.672214,96.878883,6.050838,0.556980,0.422241



Saved unified output to: test_idp_features.csv

Analysis complete! All features in one unified dataframe.
